In [2]:
import duckdb
import pandas as pd
import re
import psutil
import numpy as np
import pyfixest as pf
import gc
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
import statsmodels.formula.api as smf
import seaborn as sns
from matplotlib.lines import Line2D

pd.options.display.float_format = '{:.2f}'.format  
pd.set_option('display.max_columns', None)  # Show all columns in DataFrame output

current_dir = Path.cwd()
gitfolder = current_dir.parent.parent
datafolder = Path("//sen/project/Psonr/Data")
output = gitfolder / 'Output' 

In [3]:
#%% Load data
# Connect to DuckDB
con = duckdb.connect()

# Define file paths
base_path = Path("//sen/project/Psonr/Data/Aggregate Data")
months = ["june", "july", "aug", "sep", "oct", "nov"]

# Read the first month's data to determine column structure
sample_query = f"SELECT * FROM read_parquet('{base_path / (months[0] + '_data_week_store_sku.parquet')}') LIMIT 1"
sample_data = con.execute(sample_query).df()

# Detect column types
column_types = sample_data.dtypes.map(str).replace({
    'int64': 'BIGINT',
    'float64': 'DOUBLE',
    'object': 'VARCHAR'
}).to_dict()


# Create merged_data table with the same columns + "ppu"
columns = list(sample_data.columns) + ["ppu"]
column_types["ppu"] = "DOUBLE"

# Create table dynamically
col_str = ", ".join(f"{col} {column_types[col]}" for col in columns)
create_query = f"CREATE TABLE merged_data ({col_str})"
con.execute(create_query)

# Process each month separately and append to the table
for month in months:
    # Read data directly in DuckDB
    print(f"Processing {month}...")
    data_query = f"SELECT * FROM read_parquet('{base_path / (month + '_data_week_store_sku.parquet')}')"
    mode_query = f"SELECT week, sku_gtin, store_id, ppu FROM read_parquet('{base_path / (month + '_modes_week_store_sku.parquet')}')"

    # Merge inside DuckDB, ensuring column match
    merge_query = f"""
        INSERT INTO merged_data ({', '.join(columns)})
        SELECT d.*, m.ppu 
        FROM ({data_query}) AS d
        LEFT JOIN ({mode_query}) AS m 
        USING (week, sku_gtin, store_id)
    """
    
    con.execute(merge_query)
    print(f"Processed {month}")

df_full = con.execute("SELECT * FROM merged_data").df()



Processing june...
Processed june
Processing july...
Processed july
Processing aug...
Processed aug
Processing sep...
Processed sep
Processing oct...
Processed oct
Processing nov...
Processed nov


In [4]:
#%% Function to check memory usage
def check_memory():
    memory = psutil.virtual_memory()
    print(f"Total memory: {memory.total / (1024**3):.2f} GB")
    print(f"Available memory: {memory.available / (1024**3):.2f} GB")
    print(f"Used memory: {memory.used / (1024**3):.2f} GB")
    print(f"Memory usage: {memory.percent}%")



In [5]:
# %% Data filtering and subsampling
df_filtered = df_full[(df_full['price'] > 0) & (df_full['price'] < 100) & (df_full['quantity'] > 0) &  (df_full['quantity'] < 201)]
#df_subset['sales'] = df_subset['price'] * df_subset['quantity']

df_subset = df_filtered.sample(frac=0.1, random_state=42).copy()
del df_full, df_filtered
gc.collect()

#%%
common_sku = pd.read_csv(datafolder / 'common_skus.csv', sep=';')
#common_95 = pd.read_csv(datafolder / 'sku_kassalcat_top95.csv', sep=';')
df_subset = df_subset.merge(common_sku[['sku_gtin']], on='sku_gtin', how='inner')

In [6]:
def krone_ends_with_nine(price):
    """
    Checks if the integer part (kroner) of the price ends with 9, 
    but not if the integer part is >= 90.
    """
    if pd.isna(price):
        return False
    kroner_part = int(price)
    return kroner_part % 10 == 9 and kroner_part < 90

def ore_ends_with_nine(price):
    """
    Checks if the decimal part (øre) is exactly 0.90, 0.95, or 0.99.
    Also accepts 0.9 as 0.90.
    """
    if pd.isna(price):
        return False
    ore = round(price * 100) % 100  # Extract øre as an integer (e.g. 12.95 -> 95)
    return ore in [90, 95, 99]


In [7]:
# %% Calculate LDP
df_subset.loc[:, 'krone_ends_with_nine'] =  df_subset['ppu'].apply(krone_ends_with_nine)
df_subset.loc[:, 'ore_ends_with_nine'] = df_subset['ppu'].apply(ore_ends_with_nine)
df_subset.loc[:, 'ends_with_nine'] = df_subset['krone_ends_with_nine'] | df_subset['ore_ends_with_nine']



In [8]:
#%%
product_unique = df_subset.groupby('sku_gtin', as_index=False).agg({'store_id': 'nunique', 'week': 'nunique'})

product_nonunique = product_unique[(product_unique['store_id'] > 1) & (product_unique['week'] > 1)].sku_gtin.unique()

df_nonunique = df_subset[df_subset['sku_gtin'].isin(product_nonunique)].copy()

del df_subset
gc.collect()

check_memory()


Total memory: 150.00 GB
Available memory: 76.65 GB
Used memory: 73.35 GB
Memory usage: 48.9%


In [ ]:

df_nonunique['sku_gtin'] = df_nonunique['sku_gtin'].astype('category')
df_nonunique['week'] = df_nonunique['week'].astype('category')
df_nonunique['store_id'] = df_nonunique['store_id'].astype('category')
df_nonunique['kjedeid'] = df_nonunique['kjedeid'].astype('category')
df_nonunique['krone_ends_with_nine'] = df_nonunique['krone_ends_with_nine'].astype('int8')

In [25]:
# %%
model1 = pf.feols("krone_ends_with_nine ~ 1 | kjedeid + week + sku_gtin", data=df_nonunique)

print(model1.summary())


###

Estimation:  OLS
Dep. var.: krone_ends_with_nine, Fixed effects: kjedeid+week+sku_gtin
Inference:  
Observations:  12035588

| Coefficient   | Estimate   | Std. Error   | t value   | Pr(>|t|)   | 2.5%   | 97.5%   |
|---------------|------------|--------------|-----------|------------|--------|---------|
---

None


c:\Users\s14328\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyfixest\estimation\feols_.py:2092: UserWarning: Empty variance-covariance matrix detected
  warnings.warn(


In [26]:
# Extract FE dictionaries
fe_dict = model1.fixef()

In [27]:
# store_id FE: keys in dict are strings, so cast df to str for mapping
chain_fe_map = fe_dict["C(kjedeid)"]

df_nonunique["chain_fe"] = df_nonunique["kjedeid"].astype(str).map(chain_fe_map)

week_fe_map = fe_dict["C(week)"]

df_nonunique["week_fe"] = df_nonunique["week"].astype(str).map(week_fe_map)

sku_fe_map = fe_dict["C(sku_gtin)"]

df_nonunique["sku_fe"] = df_nonunique["sku_gtin"].astype(str).map(sku_fe_map)

In [28]:
var_p = df_nonunique["sku_fe"].var()
var_t = df_nonunique["week_fe"].var()
var_chain = df_nonunique["chain_fe"].var()
# Residuals
eps = model1.resid()    
df_nonunique['resid_step1'] = eps
var_res = eps.var()
var_D  = df_nonunique["krone_ends_with_nine"].var()
# Variance shares
shares = {
    "product": var_p / var_D,
    "week": var_t / var_D,
    "chain": var_chain / var_D,
    "residual": var_res / var_D
}
print("Variance shares:")
for k, v in shares.items():
    print(f"  {k}: {v:.4f}")

Variance shares:
  product: 0.2947
  week: 0.0001
  chain: 0.0047
  residual: 0.7003


## Regress residuals onto store FE

In [29]:
# %%
model2 = pf.feols("resid_step1 ~ 1 | store_id ", data=df_nonunique)

print(model2.summary())

###

Estimation:  OLS
Dep. var.: resid_step1, Fixed effects: store_id
Inference:  
Observations:  12035588

| Coefficient   | Estimate   | Std. Error   | t value   | Pr(>|t|)   | 2.5%   | 97.5%   |
|---------------|------------|--------------|-----------|------------|--------|---------|
---

None


c:\Users\s14328\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyfixest\estimation\feols_.py:2092: UserWarning: Empty variance-covariance matrix detected
  warnings.warn(


In [30]:
# Extract FE dictionaries
fe_dict2 = model2.fixef()

In [31]:
store_fe_map = fe_dict2["C(store_id)"]

df_nonunique["store_fe"] = df_nonunique["store_id"].astype(str).map(store_fe_map)

In [32]:

var_store = df_nonunique["store_fe"].var()
# Residuals
eps2 = model2.resid()    
df_nonunique['resid_step2'] = eps2


## Regress residual onto interacted FE (storeXproduct)

In [41]:
df_nonunique['storeXproduct'] = df_nonunique['store_id'].astype(str) + "_" + df_nonunique['sku_gtin'].astype(str)
df_nonunique['storeXproduct'] = df_nonunique['storeXproduct'].astype('category')
df_nonunique['storeXweek'] = df_nonunique['store_id'].astype(str) + "_" + df_nonunique['week'].astype(str)
df_nonunique['storeXweek'] = df_nonunique['storeXweek'].astype('category')
df_nonunique['productXweek'] = df_nonunique['sku_gtin'].astype(str) + "_" + df_nonunique['week'].astype(str)
df_nonunique['productXweek'] = df_nonunique['productXweek'].astype('category')

In [42]:
# %%
model3 = pf.feols("resid_step2 ~ 1 | storeXproduct + storeXweek + productXweek", data=df_nonunique)

print(model3.summary())

###

Estimation:  OLS
Dep. var.: resid_step2, Fixed effects: storeXproduct+storeXweek+productXweek
Inference:  
Observations:  12035588

| Coefficient   | Estimate   | Std. Error   | t value   | Pr(>|t|)   | 2.5%   | 97.5%   |
|---------------|------------|--------------|-----------|------------|--------|---------|
---

None


c:\Users\s14328\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyfixest\estimation\feols_.py:2092: UserWarning: Empty variance-covariance matrix detected
  warnings.warn(


In [43]:
# Extract FE dictionaries
fe_dict3 = model3.fixef()

In [44]:
storeXproduct_fe_map = fe_dict3["C(storeXproduct)"]
df_nonunique["storeXproduct_fe"] = df_nonunique["storeXproduct"].astype(str).map(storeXproduct_fe_map)
storeXweek_fe_map = fe_dict3["C(storeXweek)"]
df_nonunique["storeXweek_fe"] = df_nonunique["storeXweek"].astype(str).map(storeXweek_fe_map)
productXweek_fe_map = fe_dict3["C(productXweek)"]
df_nonunique["productXweek_fe"] = df_nonunique["productXweek"].astype(str).map(productXweek_fe_map)

In [45]:
var_p = df_nonunique["sku_fe"].var()
var_t = df_nonunique["week_fe"].var()
var_chain = df_nonunique["chain_fe"].var()
var_store = df_nonunique["store_fe"].var()
var_storeXproduct = df_nonunique["storeXproduct_fe"].var()
var_storeXweek = df_nonunique["storeXweek_fe"].var()
var_productXweek = df_nonunique["productXweek_fe"].var()

# Residuals
eps3 = model3.resid()    
df_nonunique['resid_step3'] = eps3
var_res = eps3.var()
var_D  = df_nonunique["krone_ends_with_nine"].var()
# Variance shares
shares = {
    "product": var_p / var_D,
    "week": var_t / var_D,
    "chain": var_chain / var_D,
    "store": var_store / var_D,
    "storeXproduct": var_storeXproduct / var_D,
    "storeXweek": var_storeXweek / var_D,
    "productXweek": var_productXweek / var_D,
    "residual": var_res / var_D
}
print("Variance shares:")
for k, v in shares.items():
    print(f"  {k}: {v:.4f}")

Variance shares:
  product: 0.2947
  week: 0.0001
  chain: 0.0047
  store: 0.0011
  storeXproduct: 0.4868
  storeXweek: 0.0042
  productXweek: 0.0814
  residual: 0.1397


### Interpretation:

- **Store × Product (0.4868)** — Almost half of the variation comes from which product is sold in which store. This means left-digit pricing is highly local and product-specific: the same product may follow different left-digit patterns across stores, and stores differ strongly in their pricing strategies across products.

- **Product (0.2947)** — Nearly 30% of variation is purely across products. Some products are systematically more likely to use 9-endings or certain left digits, regardless of store or time.

- **Product × Week (0.0814)** — Products exhibit time-varying price-ending patterns. Temporary promotions, seasonality, or product-specific pricing campaigns affect left-digit usage.

- **Chain (0.0047)** and **Store (0.0011)** — Very little variation is explained by chain-wide or store-level policies alone. Chains do not appear to have strong uniform rules about left-digit pricing. Stores on average also don’t differ much unless you look at specific products.

- **Week (0.0001)** — Almost no purely time-driven variation. There is no broad market-wide trend in left-digit usage over time.

- **Store × Week (0.0042)** — Only a tiny amount comes from store-specific dynamics over time. Stores do not strongly change left-digit practices week by week.

- **Residual (0.1397)** — Some idiosyncratic noise remains unexplained.

In [ ]:
# Extract FE dictionaries
#theta_c = fe_dict["C(kjedeid)"].values()
# theta_store = fe_dict["C(store_id)"].values()
# theta_p = fe_dict["C(sku_gtin)"].values()
# theta_t = fe_dict["C(week)"].values()

# # Residuals
# eps = model1.resid()
# eps2 = df_nonunique["krone_ends_with_nine"] - model1.predict() 

# # Variances
# var_D  = df_nonunique["krone_ends_with_nine"].var()
# #theta_c_vals = np.array(list(theta_c))
# #var_c  = np.var(theta_c_vals)
# theta_store_vals = np.array(list(theta_store))
# var_store  = np.var(theta_store_vals)
# theta_p_vals = np.array(list(theta_p))
# var_p  = np.var(theta_p_vals)
# theta_t_vals = np.array(list(theta_t))
# var_t  = np.var(theta_t_vals)
# var_e  = np.var(eps2)

# # Variance shares
# shares = {
#     #"chain":    var_c / var_D,
#     "product": var_p / var_D,
#     "week": var_t / var_D,
#     "residual": var_e / var_D,
#     "store": var_store / var_D
# }

# print("Variance shares:")
# for k, v in shares.items():
#     print(f"{k}: {v:.4f}")

Variance shares:
product: 0.5129
week: 0.0001
residual: 0.6992
store: 0.0066


In [24]:
check_memory()

Total memory: 150.00 GB
Available memory: 103.83 GB
Used memory: 46.17 GB
Memory usage: 30.8%
